In [58]:
from pycaret.regression import *

import pyodbc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from random import randint
import plotly.express as px

from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings("ignore")

In [23]:
df = pd.read_csv('C:/Users/tcardenas/OneDrive/OneDrive - Grupo DISPAPELES/Documents/ML-Dispapeles-TomasCaLo/Clustering/Clustering 1-2-23.csv', delimiter= ";")
df = df[(df["Letra cluster"] == "A") | (df["Letra cluster"] == "B")]
df_10_ag = df[df["Distrito-Nombretipozona"] == '10-Artes graficas']

In [20]:
df["Distrito-Nombretipozona"].unique()

array(['10-Artes graficas', '10-Institucional', '10-Mayorista',
       '10-V Horeca', '10-V Industria', '10-V SyE', '15-Artes graficas',
       '15-Institucional', '15-Mayorista', '17-Artes graficas',
       '17-Institucional', '17-Mayorista', '20-Artes graficas',
       '20-Institucional', '20-Mayorista', '20-V Horeca',
       '20-V Industria', '20-V SyE', '25-Artes graficas',
       '25-Institucional', '25-Mayorista', '40-Artes graficas',
       '40-Institucional', '40-Mayorista', '40-V Horeca',
       '40-V Industria', '45-Institucional', '45-Mayorista',
       '50-Artes graficas', '50-Institucional', '50-Mayorista',
       '50-V Horeca', '50-V Industria', '50-V SyE', '52-Artes graficas',
       '52-Institucional', '52-Mayorista', '53-Artes graficas',
       '53-Institucional', '53-Mayorista', '55-Artes graficas',
       '55-Institucional', '55-Mayorista', '60-Artes graficas',
       '60-Institucional', '60-Mayorista', '60-V Horeca', '60-V SyE',
       '65-Institucional', '65-Mayori

In [27]:
df.head()

,Codigo distrito,Nombre tipo zona,Nit cliente-sucursal,Recency,Monetary,Frequency,Distrito-Nombretipozona,Cluster,Letra cluster
523,10,Artes graficas,800103903-1,24,2233923,24,10-Artes graficas,Cluster 1,B
524,10,Artes graficas,800234331-1,24,1586684,22,10-Artes graficas,Cluster 1,B
525,10,Artes graficas,830068816-1,24,1576750,21,10-Artes graficas,Cluster 1,B
526,10,Artes graficas,860000198-1,24,2394980,24,10-Artes graficas,Cluster 1,B
527,10,Artes graficas,860521517-1,24,1693460,24,10-Artes graficas,Cluster 1,B


In [28]:
df_10_ag = df_10_ag.sort_values(by=["Recency", "Frequency", "Monetary"], ascending= False, )
df_10_ag

,Codigo distrito,Nombre tipo zona,Nit cliente-sucursal,Recency,Monetary,Frequency,Distrito-Nombretipozona,Cluster,Letra cluster
534,10,Artes graficas,830098200-1,24,3293858,24,10-Artes graficas,Cluster 3,A
535,10,Artes graficas,860530370-1,24,3079425,24,10-Artes graficas,Cluster 3,A
532,10,Artes graficas,800138131-1,24,2880098,24,10-Artes graficas,Cluster 3,A
526,10,Artes graficas,860000198-1,24,2394980,24,10-Artes graficas,Cluster 1,B
523,10,Artes graficas,800103903-1,24,2233923,24,10-Artes graficas,Cluster 1,B
527,10,Artes graficas,860521517-1,24,1693460,24,10-Artes graficas,Cluster 1,B
533,10,Artes graficas,800226417-1,24,4608290,23,10-Artes graficas,Cluster 3,A
529,10,Artes graficas,900085929-1,24,2165865,22,10-Artes graficas,Cluster 1,B
524,10,Artes graficas,800234331-1,24,1586684,22,10-Artes graficas,Cluster 1,B
525,10,Artes graficas,830068816-1,24,1576750,21,10-Artes graficas,Cluster 1,B


In [29]:
## Conexion al DWH
cnxn = pyodbc.connect(
    driver='{SQL Server}',
    server='192.168.100.58',
    uid='bilectura',
    pwd='D1sp@p3l3s')
cursor = cnxn.cursor()
#cnxn.close()

In [49]:
codigo_SQL = "SELECT A.DSTCOD AS 'Codigo distrito', CONCAT(CONCAT(A.VTANIT,'-'),A.VTASUC) AS 'Nit cliente-sucursal', DATEFROMPARTS(A.VTAANO, A.VTAMES, 1) AS 'Fecha', A.VTAANO AS 'Ano', A.VTAMES AS 'Mes', SUM(A.VTAVLRVTA) AS 'Ventas' FROM V_VTA_VTAHEC A LEFT JOIN DIM_CLIENTES B ON A.PRMCOD = B.PRMCOD AND A.DSTCOD = B.DSTCOD AND A.VTANIT = B.CLTNIT AND A.VTASUC = B.CLTSUC WHERE A.PRMCOD = 1 AND A.DSTCOD = 10 AND CONCAT(CONCAT(A.VTANIT,'-'),A.VTASUC) = '830098200-1' AND A.VTAFCH < DATEFROMPARTS(2023,2,1) GROUP BY A.DSTCOD,  B.VNDZNA, CONCAT(CONCAT(A.VTANIT,'-'),A.VTASUC), DATEFROMPARTS(A.VTAANO, A.VTAMES, 1), A.VTAANO, A.VTAMES"

In [127]:
#Cargue de la data desde el ERP de Dispapeles y se guarda en df
cursor.execute(codigo_SQL)
rows = cursor.fetchall()
df_SQL = pd.DataFrame.from_records(rows, columns=[col[0] for col in cursor.description])
df_SQL["Ventas"] = df_SQL["Ventas"].astype(int)

In [ ]:
# Escribir codigo SQL al dwh para traer la info de cada cliente para la regresion, primero ano mes todo lo que haya se los clientes A

In [53]:
df_SQL.columns

Index(['Codigo distrito', 'Nit cliente-sucursal', 'Fecha', 'Ano', 'Mes',
       'Ventas'],
      dtype='object')

In [128]:
df_SQL["MA12"] = df_SQL["Ventas"].rolling(3).mean()

# plot the data and MA
fig = px.line(df_SQL, x="Fecha", y=["Ventas", "MA12"])
fig.show()

In [129]:
# create a sequence of numbers
df_SQL['Series'] = np.arange(1,len(df_SQL)+1)

# drop unnecessary columns and re-arrange
df_SQL.drop(['Codigo distrito', 'MA12', 'Nit cliente-sucursal', 'Fecha'], axis=1, inplace=True)
df_SQL = df_SQL[['Series', 'Ano', 'Mes', 'Ventas']] 

# check the head of the dataset
df_SQL.head()

,Series,Ano,Mes,Ventas
0,1,2018,1,55912513
1,2,2018,2,22095734
2,3,2018,3,3730338
3,4,2018,4,663201
4,5,2018,5,56558958


In [130]:
train_n = int(round(df_SQL.shape[0] * 0.7, 0))
test_n = df_SQL.shape[0] - train_n

In [131]:
df_SQL.iloc[:train_n]

,Series,Ano,Mes,Ventas
0,1,2018,1,55912513
1,2,2018,2,22095734
2,3,2018,3,3730338
3,4,2018,4,663201
4,5,2018,5,56558958
5,6,2018,6,11119099
6,7,2018,7,18646150
7,8,2018,8,119965122
8,9,2018,9,27437796
9,10,2018,10,19977298


In [132]:
df_SQL.iloc[train_n:]

,Series,Ano,Mes,Ventas
43,44,2021,8,116390769
44,45,2021,9,201429161
45,46,2021,10,135513886
46,47,2021,11,58341346
47,48,2021,12,58424065
48,49,2022,1,51425059
49,50,2022,2,216872480
50,51,2022,3,350023322
51,52,2022,4,275499854
52,53,2022,5,164233996


In [133]:
# split data into train-test set
train = df_SQL.iloc[:train_n]
test = df_SQL.iloc[train_n:]
# check shape
train.shape, test.shape

((43, 4), (18, 4))

In [134]:
s = setup(
        data = train,
        test_data = test,
        target = 'Ventas',
        fold_strategy = 'timeseries',
        numeric_features = ['Ano', 'Series'],
        fold = 3, 
        transform_target = True,
        session_id = 42,
        silent= True,
        verbose= False
        )

In [135]:
best = compare_models(sort = 'MAE')

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
en,Elastic Net,44590642.0667,3606059674458330.5000,59876902.1237,-0.2245,1.0658,1.5781,0.0167
knn,K Neighbors Regressor,45776236.5094,3761852319731359.5000,61197759.1339,-0.2779,1.0807,1.5743,0.0233
br,Bayesian Ridge,45097499.8162,3825457084948384.0000,61695671.0698,-0.3044,1.0768,1.4492,0.0067
lasso,Lasso Regression,45329822.0000,3890116300434838.5000,62200400.5040,-0.3237,1.0772,1.4115,0.9933
ridge,Ridge Regression,49506543.8333,4039521810974816.0000,63386412.0558,-0.3756,1.2149,2.2268,0.7700
dt,Decision Tree Regressor,49271716.5667,4100976133298474.0000,63879581.5557,-0.4010,1.3154,1.7238,0.0100
rf,Random Forest Regressor,46921677.7593,4313406174594342.0000,65498456.2283,-0.4584,1.1207,1.2961,0.0533
catboost,CatBoost Regressor,45788887.8603,4477180878505153.5000,66469865.9104,-0.4735,1.0690,1.2436,0.2300
xgboost,Extreme Gradient Boosting,49907823.0583,4681238047209691.0000,68382604.4408,-0.6235,1.2072,1.2074,0.0500
llar,Lasso Least Angle Regression,47910252.9026,4695129849921553.0000,67900933.3102,-0.5355,1.1081,1.0649,0.0100


In [136]:
prediction_holdout = predict_model(best)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Elastic Net,91754381.8333,14238150798448244.0000,119323722.6977,-0.4867,0.8040,0.5963


In [137]:
# generate predictions on the original dataset
predictions = predict_model(best, data=df_SQL)

# add a date column in the dataset
predictions['Fecha'] = pd.date_range(start='2018-01-01', end = '2023-1-1', freq = 'MS')

# line plot
fig = px.line(predictions, x='Fecha', y=["Ventas", "Label"], template = 'plotly_dark')

# add a vertical rectange for test-set separation
# fig.add_vrect(x0="2018-01-01", x1="2021-8-01", fillcolor="grey", opacity=0.25, line_width=0)
fig.show()

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Elastic Net,53108871.2787,6124938284741682.0000,78261984.9272,0.0917,1.0395,1.7548


In [138]:
final_best = finalize_model(best)

In [145]:
future_dates = pd.date_range(start = '2023-02-01', end = '2023-8-01', freq = 'MS')
future_df = pd.DataFrame()
future_df['Mes'] = [i.month for i in future_dates]
future_df['Ano'] = [i.year for i in future_dates]    
future_df['Series'] = np.arange(62, (62+len(future_dates)))
future_df

,Mes,Ano,Series
0,2,2023,62
1,3,2023,63
2,4,2023,64
3,5,2023,65
4,6,2023,66
5,7,2023,67
6,8,2023,68


In [146]:
predictions_future = predict_model(final_best, data=future_df)
predictions_future

,Mes,Ano,Series,Label
0,2,2023,62,104759304.0
1,3,2023,63,106877576.0
2,4,2023,64,109026536.0
3,5,2023,65,111206376.0
4,6,2023,66,113417344.0
5,7,2023,67,115659792.0
6,8,2023,68,117933920.0


In [148]:
concat_df = pd.concat([df_SQL, predictions_future], axis=0)

concat_df_i = pd.date_range(start='2018-01-01', end = '2023-08-01', freq = 'MS')

concat_df.set_index(concat_df_i, inplace=True)

fig = px.line(concat_df, x=concat_df.index, y=["Ventas", "Label"], template = 'plotly_dark')

fig.show()